In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

########### Define your variables
beers=['Chesapeake Stout', 'Snake Dog IPA', 'Imperial Porter', 'Double Dog IPA']
ibu_values=[35, 60, 85, 75]
abv_values=[5.4, 7.1, 9.2, 4.3]
color1='darkred'
color2='orange'
mytitle='Beer Comparison'
tabtitle='beer!'
myheading='Flying Dog Beers'
label1='IBU'
label2='ABV'
githublink='https://github.com/austinlasseter/flying-dog-beers'
sourceurl='https://www.flyingdog.com/beers/'

########### Set up the chart
bitterness = go.Bar(
    x=beers,
    y=ibu_values,
    name=label1,
    marker={'color':color1}
)
alcohol = go.Bar(
    x=beers,
    y=abv_values,
    name=label2,
    marker={'color':color2}
)

beer_data = [bitterness, alcohol]
beer_layout = go.Layout(
    barmode='group',
    title = mytitle
)

beer_fig = go.Figure(data=beer_data, layout=beer_layout)


########### Initiate the app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
app.title=tabtitle

########### Set up the layout
app.layout = html.Div(children=[
    html.H1(myheading),
    dcc.Graph(
        id='flyingdog',
        figure=beer_fig
    ),
    html.A('Code on Github', href=githublink),
    html.Br(),
    html.A('Data Source', href=sourceurl),
    ]
)

if __name__ == '__main__':
    app.run_server()

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go

########### read in data
win_loss_df = pd.read_csv("win_loss_df.csv").sort_values(['wins', 'points_for'], ascending = False)

myheading = 'Raytonia Beach Fantasy Football League'

########### Set up the chart
win_bars = go.Bar(x=win_loss_df['owner_team_name'], y=win_loss_df['wins'],
                  base=0,
                  marker_color='green',
                  name='Wins',
                  hoverinfo='y'
)

loss_bars = go.Bar(x=win_loss_df['owner_team_name'], y=win_loss_df['losses'],
                   base= -1 * win_loss_df['losses'].astype('int'),
                   text = 1 * win_loss_df['losses'].astype('int'),
                   marker_color='red',
                   name='Losses',
                   hoverinfo='text'
)

win_loss_data = [win_bars, loss_bars]

#fig.update_yaxes(tickvals=[5, 4, 3, 2, 1, 0, -1, -2, -3, -4, -5], ticktext = [5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5])

win_loss_layout = go.Layout(barmode = 'overlay',
                            hovermode = 'x',
                            title = 'Wins/Losses'
)

win_loss_fig = go.Figure(data=win_loss_data, layout=win_loss_layout)

########### Initiate the app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
app.title=tabtitle


########### Set up the layout
app.layout = html.Div(children=[
    html.H1(myheading),
    dcc.Graph(
        id='flyingdog',
        figure=win_loss_fig
    )
    ]
)

if __name__ == '__main__':
    app.run_server()